# Testing LaminDB's select statements

## Mock a simple LaminDB instance

In [ ]:
!lamin init --storage sqlapi

In [ ]:
import lamindb as ln
import sqlmodel as sqm
import pandas as pd
from datetime import datetime
from typing import Union, Tuple, Optional, List

In [ ]:
class Hero(sqm.SQLModel, table=True):
    id: int = sqm.Field(primary_key=True)
    name: str
    team_id: Optional[int] = sqm.Field(foreign_key="team.id")
    team: "Team" = sqm.Relationship(back_populates="heroes")
    created_at: datetime = sqm.Field(default_factory=datetime.utcnow)

In [ ]:
class Team(sqm.SQLModel, table=True):
    id: int = sqm.Field(primary_key=True)
    name: str
    heroes: List[Hero] = sqm.Relationship(back_populates="team")

In [ ]:
# need to delete the instance here as we're not interested in the default LaminDB models
!rm sqlapi/sqlapi.lndb

In [ ]:
engine = sqm.create_engine("sqlite:///sqlapi/sqlapi.lndb")

In [ ]:
sqm.SQLModel.metadata.create_all(engine)

## Add test data

In [ ]:
with sqm.Session(engine) as session:
    team_1 = Team(id=0, name="stars")
    session.add(team_1)
    team_2 = Team(id=1, name="moons")
    session.add(team_2)
    session.add(Hero(id=0, name="test1", team_id=team_1.id))
    session.add(Hero(id=1, name="test2", team_id=team_1.id))
    session.add(Hero(id=2, name="test3"))
    session.commit()

## Test select statements

In [ ]:
ln.select(Hero, id=0).one()

In [ ]:
ln.select(Hero).df()

In [ ]:
ln.select(Hero, team_id=0).all()

In [ ]:
ln.select(Hero).where(Hero.name == "test1").df()

In [ ]:
ln.select(Hero).where(Hero.name == "test1").df()

In [ ]:
ln.select(Hero).where(Hero.id == 0, Hero.name == "test1").df()

In [ ]:
ln.select(Hero).where(Hero.id == 0, Hero.created_at <= datetime.utcnow()).df()

In [ ]:
ln.select(Hero).where(sqm.or_(Hero.name == "test1", Hero.name == "test2")).df()

In [ ]:
ln.select(Hero).where(sqm.or_(Hero.name == "test1", Hero.name == "test2")).offset(
    1
).df()

In [ ]:
ln.select(Hero).where(sqm.or_(Hero.name == "test1", Hero.name == "test2")).offset(
    0
).limit(1).df()

In [ ]:
ln.select(Hero).where(sqm.or_(Hero.name == "test1", Hero.name == "test2")).order_by(
    sqm.desc(Hero.created_at)
).df()

In [ ]:
ln.select(Hero).join(Team).df()

In [ ]:
ln.select(Hero, Team).join(Team).all()

In [ ]:
dfs = ln.select(Hero, Team).join(Team).df()

In [ ]:
for df in dfs:
    display(df)

In [ ]:
ln.select(Hero, Team).where(Hero.team_id == Team.id).all()

## Test autoflush

In [ ]:
hero = Hero(name="test4")

In [ ]:
hero

In [ ]:
assert ln.select(Hero, name="test4").one_or_none() is None

In [ ]:
team = ln.select(Team, name="stars").one()

In [ ]:
hero.team = team

In [ ]:
assert ln.select(Hero, name="test4").one_or_none() is None